In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import IPython
import random
import pandas as pd
from tqdm import tqdm
import numpy as np
def isNAN(num):
    return num != num
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
K = tf.keras.backend
import datetime
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tqdm import tqdm
import rdkit
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from math import sqrt
from rdkit.Chem.Draw import SimilarityMaps
import kerastuner as kt

def cosine_mat(x,y):
    '''x, y are same shape array'''
    if np.sum(x**2) == 0 or np.sum(y**2) == 0:
        return 0
    else:
        return (np.sum(x*y)/(sqrt(np.sum(x**2))*sqrt(np.sum(y**2))))
def isNaN(num):
    return num != num
import pickle

with open('Ontology/012921/char2idx_class_V1.pkl','rb') as f:
    class_  = pickle.load(f)
with open('Ontology/012921/char2idx_super_V1.pkl','rb') as f:
    superclass_  = pickle.load(f)
with open('Ontology/012921/char2idx_path_V1.pkl','rb') as f:
    pathway_  = pickle.load(f)

with open('ontology/012921/datset_class_all_V1.pkl','rb') as r:
    dataset = pickle.load(r)

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

def FP(smiles,radi):
    binary = np.zeros((2048*(radi)), int)
    formula = np.zeros((2048),int)
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    mol_bi = {}
    for r in range(radi+1):
        mol_fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=r, bitInfo=mol_bi, nBits = 2048)
        mol_bi_QC = []
        for i in mol_fp.GetOnBits():
            idx = mol_bi[i][0][0]
            radius_list = []
            num_ = len(mol_bi[i])
            for j in range(num_):
                if mol_bi[i][j][1] == r:
                    mol_bi_QC.append(i)
                    break

       
        if r == 0:
            for i in mol_bi_QC:
                formula[i] = len([k for k in mol_bi[i] if k[1]==0])
        else:
            for i in mol_bi_QC:
                binary[(2048*(r-1))+i] = len([k for k in mol_bi[i] if k[1]==r])
    
    
    
    return formula.reshape(1,2048),binary.reshape(1,4096)



def dataset_generation(idx):
    X_train_f = np.zeros((len(idx),2048),int)
    X_train_b = np.zeros((len(idx),4096),int)
    Y_train_path = np.zeros((len(idx),len(pathway_)),int)
    Y_train_super = np.zeros((len(idx),len(superclass_)),int)
    Y_train_class = np.zeros((len(idx),len(class_)),int)
    for i,n in enumerate(idx):
        smiles = dataset[n]['SMILES']
        X_train_f[i] = FP(smiles,2)[0]
        X_train_b[i] = FP(smiles,2)[1]
        #Y_train_path[i] = dataset[n]['Pathway']
        #Y_train_super[i] = dataset[n]['Super_class']
        Y_train_class[i] = dataset[n]['Class']
        
    return [X_train_f,X_train_b], Y_train_class

def model_build_DO(hp):
    hp_unit1 = hp.Int('unit1', min_value = 3072, max_value = 6144, step = 1536) #3
    hp_unit2 = hp.Int('unit2', min_value = 1536, max_value = 3072, step = 768) #3
    hp_unit3 = hp.Int('unit3', min_value = 768, max_value = 1536, step = 384) #3
    hp_drop = hp.Int('unit4', min_value = 1, max_value = 3, step = 1)/10 #3
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
    hp_l2_rate = hp.Choice('regularizer', values = [1e-1, 1e-2, 1e-3, 1e-4])
    
    l2 = keras.regularizers.l2(hp_l2_rate)
    
    
    input_f = layers.Input(shape=(2048,))
    input_b = layers.Input(shape=(4096,))
    input_fp = layers.Concatenate()([input_f,input_b])
    
    X = layers.Dense(6144, activation = 'relu', kernel_regularizer=l2)(input_fp)
    X = layers.BatchNormalization()(X)
    X = layers.Dense(hp_unit1, activation = 'relu', kernel_regularizer=l2)(X)
    X = layers.BatchNormalization()(X)
    X = layers.Dense(hp_unit2, activation = 'relu', kernel_regularizer=l2)(X)
    X = layers.BatchNormalization()(X)
    X = layers.Dense(hp_unit3, activation = 'relu', kernel_regularizer=l2)(X)
    X = layers.Dropout(hp_drop)(X)
    output = layers.Dense(len(class_), activation = 'sigmoid')(X)
    model = keras.Model(inputs = [input_f,input_b], outputs = output)
    model.compile(optimizer=keras.optimizers.Adam(lr=hp_learning_rate, decay=1e-6),loss=['binary_crossentropy'],metrics=['cosine_proximity',top_k_categorical_accuracy])
    return model  


def top_k_categorical_accuracy(y_true, y_pred):
    return tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=2)


class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

np.random.seed(0)

b_key = list(dataset.keys())
np.random.shuffle(b_key)
dict_ = np.array(b_key)
Y_ = np.array([ np.max(np.where(dataset[i]['Class']==1)[0]) for i in dict_])

train_D, test_dict, y_train, y_test = train_test_split(dict_, Y_, test_size=0.2, random_state=1, stratify = Y_)
train_dict, val_dict, y_train, y_val = train_test_split(train_D, y_train, test_size=0.2, random_state=1, stratify = y_train)


In [4]:
X_train, Y_train = dataset_generation(train_dict)
X_val, Y_val = dataset_generation(val_dict)

In [6]:
tuner = kt.Hyperband(model_build_DO,
                     objective = 'val_loss', 
                     max_epochs = 200,
                     factor = 3,
                     directory = 'my_dir2',
                     project_name = 'npc_class')

In [ ]:
tuner.search(x=X_train, y=Y_train,batch_size=128, epochs=200, validation_data=(X_val,Y_val), callbacks=[ClearTrainingOutput()])